# ACTIVITY SUGGESTION 



## Students
    
    18127183 - Tran Bao Phuc


## Introduction

https://www.boredapi.com/
The Bored API helps you find things to do when you're bored! There are fields like the number of participants, activity type, and more that help you narrow down your results.

Website return a random activity within a simple request, example:

http://www.boredapi.com/api/activity/

```json
    {
        "activity": "Learn Express.js",
        "accessibility": 0.25,
        "type": "education",
        "participants": 1,
        "price": 0.1,
        "link": "https://expressjs.com/",
        "key": "3943506"
    } 
```

We need to understand the fields's descriptions are:
- `activity`: Description of the queried activity
- `accessibility`: A factor describing how possible an event is to do with zero being the most accessible [0.0, 1.0]
- `type`: Description of the queried activity ["education", "recreational", "social", "diy", "charity", "cooking", "relaxation", "music", "busywork"]
- `participants`: The number of people that this activity could involve [0, n]
- `price`: A factor describing the cost of the event with zero being free
- `key`: A unique numeric id


## Scope

For this project we, students will get random activity suggestion from the service. Then we will program model to predict `accessibility` based on certain factors.



## Setup


For this we will use `min-ds_env` as environment
To verify:

In [1]:
import sys
sys.executable

'/Users/phuccoker/opt/miniconda3/envs/min_ds-env/bin/python'

#### Import

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, 
                      # giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Data collection

In [3]:
import requests
import csv
import json
import time

A function that send get requests up to `n` time and save response data to the `csv` file
`interval`: sleep time between request
`amount`: amount of requests

In [4]:
csv_file = './data.csv'
url = "http://www.boredapi.com/api/activity"
colnames=['key', 'activity', 'accessibility', 'type', 'participants', 'price', 'link']

In [5]:
 
def collectData(amount=2, interval=1):
    f = csv.DictWriter(open(csv_file, "w+"), fieldnames=colnames)
    for i in range(amount):
        time.sleep(interval)
        response = requests.get(url)
        res = response.json()
        f.writerow(res)

In [6]:
# collectData(100015, 1)

We collected 10015 record in `data.csv`

### Data Discovering

In [7]:

data_df = pd.read_csv(csv_file, names=colnames, 
                      index_col=0) # index column: "key"
data_df.head()


,activity,accessibility,type,participants,price,link
key,,,,,,
4558850,Go to a concert with some friends,0.40,social,4,0.60,NaN
2237769,Explore the nightlife of your city,0.32,social,1,0.10,NaN
1799120,Cook something together with someone,0.80,cooking,2,0.30,NaN
4614092,Draw and color a Mandala,0.10,relaxation,1,0.05,https://en.wikipedia.org/wiki/Mandala
9216391,Learn woodworking,0.30,diy,1,0.30,NaN


In [8]:
data_df.shape

(10015, 6)

Because data is randomize for every request, we check if any duplicated rows

In [9]:
data_df.duplicated().sum()

9819

Wow, too many duplicated rows!!!

In [10]:
data_df = data_df.drop_duplicates(keep='first')

In [11]:
data_df.shape

(196, 6)

In [12]:
### preprocess data (Raw)


In [13]:
y_sr = data_df["accessibility"]
X_df = data_df.drop(["accessibility"], axis=1)

In [14]:
y_sr

key
4558850    0.40
2237769    0.32
1799120    0.80
4614092    0.10
9216391    0.30
           ... 
3646173    0.00
2581372    0.10
2085321    0.30
9026787    0.10
6813070    0.10
Name: accessibility, Length: 196, dtype: float64

In [15]:
X_df

,activity,type,participants,price,link
key,,,,,
4558850,Go to a concert with some friends,social,4,0.60,NaN
2237769,Explore the nightlife of your city,social,1,0.10,NaN
1799120,Cook something together with someone,cooking,2,0.30,NaN
4614092,Draw and color a Mandala,relaxation,1,0.05,https://en.wikipedia.org/wiki/Mandala
9216391,Learn woodworking,diy,1,0.30,NaN
...,...,...,...,...,...
3646173,Learn Morse code,education,1,0.00,https://en.wikipedia.org/wiki/Morse_code
2581372,Take a bubble bath,relaxation,1,0.15,NaN
2085321,Take a spontaneous road trip with some friends,social,4,0.20,NaN


In [16]:
train_X_df, val_X_df, train_y_sr, val_y_sr = \
                              train_test_split(X_df, y_sr, 
                                               test_size=0.3, 
                                               random_state=0)

In [17]:
train_X_df.head().index

Int64Index([5881028, 6778219, 6808057, 4290333, 4266522], dtype='int64', name='key')

In [18]:
### Discovering data

First, we have a look at data types of each fields

In [19]:
train_X_df.dtypes

activity         object
type             object
participants      int64
price           float64
link             object
dtype: object

In [20]:
def missing_percentage(c):
    return (c.isna().mean() * 100).round(1)

In [21]:
train_X_df.agg([missing_percentage])

,activity,type,participants,price,link
missing_percentage,0.0,0.0,0.0,0.0,88.3


Only column `link` has missing values. Other columns does not.

How many types of activity do we get?

In [22]:
train_X_df['type'].value_counts()

recreational    31
social          29
busywork        22
education       17
relaxation      15
charity          7
music            6
diy              5
cooking          5
Name: type, dtype: int64

In [23]:
### Preprocess data (Train set)

- col 'activity' => number of words (more words means more complicating the activity)
- col 'type' => one hot encoding 
- col 'link' => number of 0 and 1 (0 for empty, 1 for any value)

In [24]:
class ColPreprocess(BaseEstimator, TransformerMixin):
    def __init__(self):
       pass

    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        df = X_df.copy()
        df['activity'] = [len(x.split()) for x in X_df['activity']]
        df['link'] = [1 if x else 0 for x in X_df['link']]
        return df

# # TEST Transform METHOD
col_prep = ColPreprocess()
col_prep.transform(train_X_df)

,activity,type,participants,price,link
key,,,,,
5881028,5,education,1,0.10,1
6778219,6,education,1,0.00,1
6808057,6,recreational,1,0.00,1
4290333,8,relaxation,1,0.10,1
4266522,4,busywork,1,0.50,1
...,...,...,...,...,...
6693574,6,recreational,1,0.10,1
2581372,4,relaxation,1,0.15,1
4688012,2,recreational,1,0.00,1


In [25]:
col_preprocess2 = ColumnTransformer([
    ("onehot", OneHotEncoder(handle_unknown="ignore"), ['type'])
])

preprocess_pipeline = Pipeline([
    ('col_preprocess', ColPreprocess()),
    ('col_preprocess2', col_preprocess2),
    ('normalize', StandardScaler(with_mean=False)),
])



In [26]:
preprocess_pipeline

Pipeline(steps=[('col_preprocess', ColPreprocess()),
                ('col_preprocess2',
                 ColumnTransformer(transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type'])])),
                ('normalize', StandardScaler(with_mean=False))])

In [27]:
preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)